In [ ]:
#upload some libraries
import os
import pandas.testing as tm
import h5py
import numpy as np
import pandas as pd
import math
import sys
import subprocess
import statsmodels.api as sm
from scipy import stats
import statsmodels.formula.api as smf
from patsy.builtins import *
import statistics
#import umap.umap_ as umap
#import umap as umap
from joblib import dump, load
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.decomposition import PCA

! pip install --upgrade tables
import tables

# set paths
basedir = '/data/songy4/tes'
datadir = f'{basedir}/data_folder'
workdir = f'{datadir}/output'

Defaulting to user installation because normal site-packages is not writeable


In [4]:
#download the renormalized datasets ---- run it only once
#TElocal output files normalized with limma for a different timepoint, weren't adjusted for any covariates
! cp -R /data/LNG/Frank/TE/limma/baseline_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v04_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v06_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v08_GENE_TE_norm_adjusted.cntTable /{datadir}/

#TE location data
! cp -R /data/LNG/Frank/TE/hg38_rmsk_TE.gtf.locInd.locations /{datadir}/

#gene locations
! cp -R /data/LNG/Frank/TE/hg38_gene_annotation.gtf.locations /{datadir}/

#plate information (one of covariates among phenotype (= case vs control), sex, age, plate, expression pcs 1 to 5)
! cp -R /data/LNG/Frank/TE/TElocal_replicates/megaMetaTable.csv /{datadir}/

#genomic data (plink files)
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.bed /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.bim /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.fam /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.log /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.nosex /{datadir}/

#PD GWAS sum stats
! cp -R /data/LNG/Frank/meta5_sumstats_harmonized_hg19_hg38.csv /{datadir}/

#pheno
! cp -R /data/LNG/Frank/TE/DEMOG_DIAG.csv /{datadir}/

#/data/CARD/PD/summary_stats --> PD GWAS sum stats were based on files from here

GWAS summary stats

In [3]:
#open pheno file megaMetaTable.csv
statsum = pd.read_csv(f"{datadir}/meta5_sumstats_harmonized_hg19_hg38.csv", index_col=False )

print('shape of GWAS summary stats data:', statsum.shape)
statsum

shape of GWAS summary stats data: (7818616, 14)


,RSID,A1,A2,REF,ALT,BETA,SE,FREQ,P,LOG_P,CHR,BP_HG19,CHR_BP_HG19_REF_ALT,BP_HG38
0,rs7899632,A,G,A,G,0.0110,0.0095,0.5665,0.2476,0.606249,10.0,100000625.0,10:100000625_A/G,98240868.0
1,rs61875309,A,C,A,C,-0.0091,0.0116,0.7953,0.4295,0.367037,10.0,100000645.0,10:100000645_A/C,98240888.0
2,rs150203744,T,C,C,T,-0.0152,0.0649,0.0140,0.8147,0.089002,10.0,100001867.0,10:100001867_C/T,98242110.0
3,rs111551711,T,C,T,C,0.0347,0.0742,0.9868,0.6396,0.194092,10.0,100002464.0,10:100002464_T/C,98242707.0
4,rs12258651,T,G,T,G,-0.0011,0.0149,0.8819,0.9423,0.025811,10.0,100003242.0,10:100003242_T/G,98243485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7818611,rs11794422,T,C,C,T,-0.0035,0.0117,0.4523,0.7663,0.115601,9.0,99997707.0,9:99997707_C/T,97235425.0
7818612,rs10981296,T,C,C,T,-0.0037,0.0114,0.4525,0.7462,0.127145,9.0,99998403.0,9:99998403_C/T,97236121.0
7818613,rs10981297,C,G,C,G,0.0031,0.0148,0.8132,0.8333,0.079199,9.0,99998646.0,9:99998646_C/G,97236364.0
7818614,rs151001359,C,G,G,C,-0.0173,0.1622,0.0142,0.9153,0.038437,9.0,9999880.0,9:9999880_G/C,9999880.0


In [5]:
#flip Allele1 and Allele2 if Freq1 is bigger than .5 and change Freq1 to negative
stat_sum = statsum.copy()
cond = (stat_sum['FREQ'] > 0.5)
stat_sum.loc[cond, ['A1', 'A2']] = stat_sum.loc[cond, ['A2', 'A1']].values
#stat_sum_flip.Allele1, stat_sum_flip.Allele2 = np.where(stat_sum_flip.Freq1 > 0.5, [stat_sum_flip.Allele2, stat_sum_flip.Allele1], [stat_sum_flip.Allele1, stat_sum_flip.Allele2])

#change Freq1
stat_sum.loc[cond, ['FREQ']] = 1-stat_sum['FREQ']

#change Effect value
stat_sum.loc[cond, 'BETA'] *= -1

stat_sum
##no need to change  StdErr, P-value

,RSID,A1,A2,REF,ALT,BETA,SE,FREQ,P,LOG_P,CHR,BP_HG19,CHR_BP_HG19_REF_ALT,BP_HG38
0,rs7899632,G,A,A,G,-0.0110,0.0095,0.4335,0.2476,0.606249,10.0,100000625.0,10:100000625_A/G,98240868.0
1,rs61875309,C,A,A,C,0.0091,0.0116,0.2047,0.4295,0.367037,10.0,100000645.0,10:100000645_A/C,98240888.0
2,rs150203744,T,C,C,T,-0.0152,0.0649,0.0140,0.8147,0.089002,10.0,100001867.0,10:100001867_C/T,98242110.0
3,rs111551711,C,T,T,C,-0.0347,0.0742,0.0132,0.6396,0.194092,10.0,100002464.0,10:100002464_T/C,98242707.0
4,rs12258651,G,T,T,G,0.0011,0.0149,0.1181,0.9423,0.025811,10.0,100003242.0,10:100003242_T/G,98243485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7818611,rs11794422,T,C,C,T,-0.0035,0.0117,0.4523,0.7663,0.115601,9.0,99997707.0,9:99997707_C/T,97235425.0
7818612,rs10981296,T,C,C,T,-0.0037,0.0114,0.4525,0.7462,0.127145,9.0,99998403.0,9:99998403_C/T,97236121.0
7818613,rs10981297,G,C,C,G,-0.0031,0.0148,0.1868,0.8333,0.079199,9.0,99998646.0,9:99998646_C/G,97236364.0
7818614,rs151001359,C,G,G,C,-0.0173,0.1622,0.0142,0.9153,0.038437,9.0,9999880.0,9:9999880_G/C,9999880.0


In [7]:
#drop two rows Freq1 is smaller than 0.01
stat_sum.drop(stat_sum[stat_sum.FREQ < 0.01].index, inplace=True)
stat_sum = stat_sum.reset_index(drop=True)
#stat_sum = stat_sum.drop(['index'], axis=1)

#drop unwanted columns
stat_sum = stat_sum[['RSID', 'REF', 'ALT', 'BETA', 'SE', 'P','BP_HG19']]

#check the unique number of MarkerName in meta
print("Unique RSID in meta data:", stat_sum['RSID'].nunique())

stat_sum

Unique RSID in meta data: 7781721


,RSID,REF,ALT,BETA,SE,P,BP_HG19
0,rs7899632,A,G,-0.0110,0.0095,0.2476,100000625.0
1,rs61875309,A,C,0.0091,0.0116,0.4295,100000645.0
2,rs150203744,C,T,-0.0152,0.0649,0.8147,100001867.0
3,rs111551711,T,C,-0.0347,0.0742,0.6396,100002464.0
4,rs12258651,T,G,0.0011,0.0149,0.9423,100003242.0
...,...,...,...,...,...,...,...
7818609,rs11794422,C,T,-0.0035,0.0117,0.7663,99997707.0
7818610,rs10981296,C,T,-0.0037,0.0114,0.7462,99998403.0
7818611,rs10981297,C,G,-0.0031,0.0148,0.8333,99998646.0
7818612,rs151001359,G,C,-0.0173,0.1622,0.9153,9999880.0


In [9]:
#print duplicated rows
stat_sum_dup = stat_sum[stat_sum['RSID'].duplicated(keep=False)]
print(stat_sum_dup.head())

#count how many times patient number is duplicated
duplicate = stat_sum_dup.groupby(stat_sum_dup['RSID'].tolist(),as_index=False).size()
print(duplicate)

#drop rows with RSID as .
stat_sum = stat_sum[stat_sum.RSID != '.']

#drop duplicated ID into 1 
stat_sum = stat_sum.drop_duplicates(['RSID']).reset_index(drop=True)

#calculate z score with effect size and standard error (z = (score-mean)/se)
def z_score(a, b):
    if b == 0:
        return 0.0
    return float(a)/b

stat_sum['Z'] = np.vectorize(z_score)(stat_sum['BETA'], stat_sum['SE'])
stat_sum

,RSID,REF,ALT,BETA,SE,P,BP_HG19,Z
248,rs7076493,C,T,0.0139,0.0201,0.4877,100082385.0,0.691542
249,rs7076493,C,A,0.0139,0.0201,0.4877,100082385.0,0.691542
250,rs7076493,C,T,0.0139,0.0201,0.4877,100082385.0,0.691542
251,rs7076493,C,A,0.0139,0.0201,0.4877,100082385.0,0.691542
656,rs11592273,C,T,-0.0037,0.0198,0.8536,100189568.0,-0.186869
...,...,...,...,...,...,...,...,...
7818124,.,NaN,NaN,-0.1323,0.2083,0.5254,NaN,-0.635142
7818177,rs73547538,A,G,-0.0402,0.1835,0.8265,99834803.0,-0.219074
7818178,rs73547538,A,C,-0.0402,0.1835,0.8265,99834803.0,-0.219074
7818179,rs73547538,A,G,-0.0402,0.1835,0.8265,99834803.0,-0.219074


In [15]:
#calculate chisq
stat_sum['chisq'] = stat_sum['Z']**2

#calculate lambda (median(z^2) / 0.455)
lambda_statsum = ((stat_sum['P'].median())/0.456)
print(lambda_statsum)
stat_sum

1.0210526315789474


,RSID,REF,ALT,BETA,SE,P,BP_HG19,Z,chisq
0,rs7899632,A,G,-0.0110,0.0095,0.2476,100000625.0,-1.157895,1.340720
1,rs61875309,A,C,0.0091,0.0116,0.4295,100000645.0,0.784483,0.615413
2,rs150203744,C,T,-0.0152,0.0649,0.8147,100001867.0,-0.234206,0.054853
3,rs111551711,T,C,-0.0347,0.0742,0.6396,100002464.0,-0.467655,0.218701
4,rs12258651,T,G,0.0011,0.0149,0.9423,100003242.0,0.073826,0.005450
...,...,...,...,...,...,...,...,...,...
7781715,rs11794422,C,T,-0.0035,0.0117,0.7663,99997707.0,-0.299145,0.089488
7781716,rs10981296,C,T,-0.0037,0.0114,0.7462,99998403.0,-0.324561,0.105340
7781717,rs10981297,C,G,-0.0031,0.0148,0.8333,99998646.0,-0.209459,0.043873
7781718,rs151001359,G,C,-0.0173,0.1622,0.9153,9999880.0,-0.106658,0.011376


In [17]:
#rename columns 
sumstat = stat_sum.copy()
sumstat.rename(columns={'RSID':'SNP', 'REF':'A1', 'ALT':'A2'}, inplace=True)

#drop BETA, SE, P, BP_HG19, chisq colunms 
sumstat = sumstat.drop(['BETA', 'SE', 'P', 'BP_HG19', 'chisq'], axis=1)

sumstat = sumstat.reset_index(drop=True)
print("shape of meta data:", sumstat.shape)
sumstat.head()

shape of meta data: (7781720, 4)


,SNP,A1,A2,Z
0,rs7899632,A,G,-1.157895
1,rs61875309,A,C,0.784483
2,rs150203744,C,T,-0.234206
3,rs111551711,T,C,-0.467655
4,rs12258651,T,G,0.073826


In [18]:
#save meta_df as txt -- run only once for download
sumstat.to_csv(r'./data_folder/meta.txt', sep='\t' ,index=False)
stat_sum.to_csv(r'./data_folder/meta_detail.txt', sep='\t' ,index=False)